In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, SGDRegressor
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, normalize
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
import pickle
from tqdm import tqdm
import math
# from tsmoothie.smoother import KalmanSmoother
# sklearn.linear_model.SGDRegressor

In [ ]:
crypto = "matic"
without_meta = 1
save = False

In [ ]:
if(without_meta == 1):
  # results_path = f"/content/drive/MyDrive/IR_Project/final_sentiment_tweet_crypto_data/LSTM_Results/{crypto}/without_meta/"
  results_path = f"/content/drive/MyDrive/IR_Project/final_sentiment_tweet_crypto_data/final_LSTM_results/{crypto}/without_meta/"
else:
  # results_path = f"/content/drive/MyDrive/IR_Project/final_sentiment_tweet_crypto_data/LSTM_Results/{crypto}/with_meta/"
  results_path = f"/content/drive/MyDrive/IR_Project/final_sentiment_tweet_crypto_data/final_LSTM_results/{crypto}/with_meta/"

In [ ]:
sentiment_df = pd.read_csv(f"/content/drive/MyDrive/IR_Project/final_sentiment_tweet_crypto_data/{crypto}_sentiment.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,6,8,10,11,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
sentiment_df.shape

(263119, 23)

In [ ]:
sentiment_df.loc[sentiment_df["user_verified"] == 'True', "user_verified"] = 1
sentiment_df.loc[sentiment_df["user_verified"] == 'False', "user_verified"] = -1

In [ ]:
sentiment_df["positive_score"] = 0
sentiment_df["negative_score"] = 0
sentiment_df["neutral_score"] = 0
sentiment_df["scaled_score"] = 0

In [ ]:
import random

In [ ]:
def scale_sentiment_score(label, score):
  if(label == 'positive'):
    # print('pos')
    scaled_score = score
  elif(label == 'negative'):
    scaled_score = -1 * score
  else:
    scaled_score = random.choice([-1, 1])*(0.5 - (0.5 * score))
  return scaled_score

sentiment_df['scaled_score'] = sentiment_df.apply(lambda x: scale_sentiment_score(x.sentiment_label, x.sentiment_score), axis=1)

In [ ]:
# for i in tqdm(range(sentiment_df.shape[0])):
#   scaled_score = 0
#   label = sentiment_df.iloc[i]['sentiment_label']
#   score = sentiment_df.iloc[i]['sentiment_score']
#   if(label == 'positive'):
#     scaled_score = score
#   elif(label == 'negative'):
#     scaled_score = -1 * score
#   else:
#     scaled_score = random.choice([-1, 1])*(0.5 - (0.5 * score))
#   sentiment_df.loc[i, 'scaled_score'] = scaled_score

In [ ]:
sentiment_df['positive_score'] = np.where(sentiment_df.sentiment_label == 'positive', sentiment_df.sentiment_score, 0)
sentiment_df['negative_score'] = np.where(sentiment_df.sentiment_label == 'negative', sentiment_df.sentiment_score, 0)
sentiment_df['neutral_score'] = np.where(sentiment_df.sentiment_label == 'neutral', sentiment_df.sentiment_score, 0)

In [ ]:
sentiment_df['scaled_score']

0        -0.109407
1         0.059999
2        -0.157771
3        -0.509790
4        -0.126895
            ...   
263114    0.034388
263115    0.978472
263116   -0.040370
263117   -0.045906
263118   -0.148154
Name: scaled_score, Length: 263119, dtype: float64

In [ ]:
sentiment_df['avg_price'] = (sentiment_df.high + sentiment_df.low)/2

In [ ]:
sentiment_df_copy = sentiment_df.copy(deep=True)

In [ ]:
enc = OneHotEncoder(sparse=False)
color_onehot = enc.fit_transform(sentiment_df_copy[['sentiment_label']])

In [ ]:
sentiment_labels = pd.DataFrame(color_onehot, columns=list(enc.categories_[0]))
# verif_labels = pd.DataFrame(verif_onehot, columns=['not_verified', 'verified'])
# print(verif_labels)
sentiment_df_copy = pd.concat([sentiment_df_copy, sentiment_labels], axis=1)

In [ ]:
sentiment_df_copy.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'index', 'id', 'text', 'favourite_count',
       'retweet_count', 'created_at', 'user_id_str', 'user_name',
       'user_followers_count', 'user_verified', 'hashtags', 'mentioned',
       'high', 'low', 'open', 'volume_to', 'volume_from', 'close',
       'clean_text', 'sentiment_label', 'sentiment_score', 'positive_score',
       'negative_score', 'neutral_score', 'scaled_score', 'avg_price',
       'negative', 'neutral', 'positive'],
      dtype='object')

In [ ]:
# sentiment_df_copy.drop(columns=['sentiment_score'])
sentiment_df_copy = sentiment_df_copy.reset_index()
sentiment_df_copy.dropna(inplace=True)

In [ ]:
sentiment_df_copy['created_at'] = pd.to_datetime(sentiment_df_copy['created_at'])

In [ ]:
sentiment_df_copy['year'] = sentiment_df_copy['created_at'].dt.year
sentiment_df_copy['month'] = sentiment_df_copy['created_at'].dt.month
sentiment_df_copy['day'] = sentiment_df_copy['created_at'].dt.day
sentiment_df_copy['hour'] = sentiment_df_copy['created_at'].dt.hour
sentiment_df_copy['minute'] = sentiment_df_copy['created_at'].dt.minute
sentiment_df_copy['second'] = sentiment_df_copy['created_at'].dt.second

In [ ]:
# sentiment_df_copy['']

In [ ]:
sentiment_df_copy['sentiment_val'] = 0

In [ ]:
def apply_mapping(x):
  if(x == 'positive'):
    return 1
  elif(x == 'negative'):
    return -1
  elif(x == 'neutral'):
    return 0

# def apply_verif_
  
sentiment_df_copy['sentiment_mapping'] = sentiment_df_copy['sentiment_label'].apply(apply_mapping)

In [ ]:
# import math
# def apply_log(x):
#   # print(x)
#   if(float(x) == 0):
#     return 0
#   return math.log10(float(x))
# # print(apply_log(100))

# sentiment_df_copy['log_rt'] = sentiment_df_copy['retweet_count'].apply(apply_log)
# sentiment_df_copy['log_fav'] = sentiment_df_copy['favourite_count'].apply(apply_log)
# # sentiment_df_copy['log_']

In [ ]:
mm = MinMaxScaler()
sentiment_df_copy[['popularity_scale', 'favourite_scale', 'retweet_scale']] = mm.fit_transform(sentiment_df_copy[['user_followers_count', 'favourite_count', 'retweet_count']])

In [ ]:
# list_of_features = ['favourite_count', 'retweet_count', 'user_followers_count', 'user_verified', 'negative', 'neutral', 'positive', 'positive_score', 'negative_score', 'neutral_score', 'high', 'low', 'open', 'volume_to', 'volume_from', 'close']
# list_of_features = ['log_rt', 'log_fav', 'sentiment_mapping', 'scaled_score', 'high', 'low', 'open', 'volume_to', 'volume_from', 'close']
if(without_meta == 1):
  list_of_features = ['scaled_score', 'sentiment_mapping', 'high', 'low', 'open', 'volume_to', 'volume_from', 'close']
else:
  list_of_features = ['popularity_scale', 'favourite_scale', 'retweet_scale', 'user_verified', 'scaled_score', 'sentiment_mapping', 'high', 'low', 'open', 'volume_to', 'volume_from', 'close']
# list_of_features = ['favourite_count', 'retweet_count', 'user_followers_count', 'user_verified', 'negative', 'neutral', 'positive', 'positive_score', 'negative_score', 'neutral_score', 'high', 'low', 'open', 'volume_to', 'volume_from', 'close', 'avg_price']


In [ ]:
model_data = np.array(sentiment_df_copy[list_of_features])
model_data.shape

(263077, 8)

In [ ]:
sent = sentiment_df_copy[['sentiment_score', 'sentiment_mapping', 'scaled_score','close','open', 'high', 'low']]
sent.corr(method ='pearson')

,sentiment_score,sentiment_mapping,scaled_score,close,open,high,low
sentiment_score,1.000000,0.064830,0.138106,0.024170,0.024124,0.024160,0.024137
sentiment_mapping,0.064830,1.000000,0.953331,0.057182,0.057218,0.057243,0.057170
scaled_score,0.138106,0.953331,1.000000,0.059861,0.059879,0.059913,0.059842
close,0.024170,0.057182,0.059861,1.000000,0.999854,0.999929,0.999936
open,0.024124,0.057218,0.059879,0.999854,1.000000,0.999933,0.999922
high,0.024160,0.057243,0.059913,0.999929,0.999933,1.000000,0.999900
low,0.024137,0.057170,0.059842,0.999936,0.999922,0.999900,1.000000


In [ ]:
if(crypto == 'avax'):
    model_data = np.delete(model_data, 20642, 0)

In [ ]:
model_data = model_data.astype(float)

In [ ]:
model_data.shape

(263077, 8)

In [ ]:
num_feats = model_data.shape[1] - 1

In [ ]:
feature_data = model_data[:, :num_feats]
print(feature_data.shape)
y_price = model_data[:, num_feats:]
print(y_price.shape)

(263077, 7)
(263077, 1)


In [ ]:
ss = StandardScaler()
# smoother = KalmanSmoother(component='level_longseason', 
#                           component_noise={'level':0.1, 'longseason':0.1}, 
#                           n_longseasons=365)
# X_ss = ss.fit_transform(feature_data)
# y_mm = mm.fit_transform(y_price) 

X_ss = feature_data
y_mm = y_price

In [ ]:
test_size = 0.20
total_samples = X_ss.shape[0]
test_samples = int(test_size * total_samples)
train_samples = total_samples - test_samples

X_train = X_ss[:train_samples, :]
X_test = X_ss[train_samples:,:]
y_train = y_mm[:train_samples]
y_test = y_mm[train_samples:]

In [ ]:
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

# smoother.smooth(X_train)
# smoother.smooth(X_test)

y_train = mm.fit_transform(y_train)
y_test = mm.transform(y_test)
# X_train = normalize(X_train, norm='l2')
# X_test = normalize(X_test, norm='l2')


In [ ]:
print("Training Shape", X_train.shape, y_train.shape)
print("Testing Shape", X_test.shape, y_test.shape) 

Training Shape (210462, 7) (210462, 1)
Testing Shape (52615, 7) (52615, 1)


In [ ]:
import torch #pytorch
import torch.nn as nn
from torch.autograd import Variable 

In [ ]:
X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test)) 

In [ ]:
X_train_tensors_final = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))
X_test_tensors_final = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], 1, X_test_tensors.shape[1])) 

In [ ]:
print("Training Shape", X_train_tensors_final.shape, y_train_tensors.shape)
print("Testing Shape", X_test_tensors_final.shape, y_test_tensors.shape) 

Training Shape torch.Size([210462, 1, 7]) torch.Size([210462, 1])
Testing Shape torch.Size([52615, 1, 7]) torch.Size([52615, 1])


In [ ]:
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(num_layers *hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.05)
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size * self.num_layers) #reshaping the data for Dense layer next
        # out = self.relu(hn)
        out = self.dropout(hn)
        out = self.fc_1(hn) #first Dense
        out = self.relu(out) #relu
        out = self.dropout(out)
        out = self.fc(out) #Final Output
        out = self.sigmoid(out)
        return out

In [ ]:
num_epochs = 200 #1000 epochs
learning_rate = 8e-4 #0.001 lr

input_size = num_feats #number of features
hidden_size = 16 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 1 #number of output classes 

In [ ]:
lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1]) #our lstm class 

In [ ]:
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate, betas=(0.9, 0.999), weight_decay=0.01)
# optimizer = torch.optim.SGD(lstm1.parameters(), lr=learning_rate, momentum=0.05, weight_decay=0.01)

In [ ]:
train_loss = []
val_loss = []

for epoch in range(num_epochs):

  optimizer.zero_grad() #caluclate the gradient, manually setting to 0
  outputs = lstm1(X_train_tensors_final) #forward pass
  # optimizer.zero_grad() #caluclate the gradient, manually setting to 0
 
  # obtain the loss function
  loss = criterion(outputs, y_train_tensors)
  train_loss.append(loss.item()/len(y_train_tensors))
  loss.backward() #calculates the loss of the loss function
 
  optimizer.step() #improve from loss, i.e backprop
  print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 
  output2 = lstm1(X_test_tensors_final) #forward pass
  #   # optimizer.zero_grad() #caluclate the gradient, manually setting to 0
  #   # obtain the loss function
  loss = criterion(output2, y_test_tensors)
  print("Validation Loss: ", loss.item()) 
  val_loss.append(loss.item()/len(y_test_tensors))

Epoch: 0, loss: 0.08646
Validation Loss:  0.18942153453826904
Epoch: 1, loss: 0.08568
Validation Loss:  0.19040313363075256
Epoch: 2, loss: 0.08490
Validation Loss:  0.19138523936271667
Epoch: 3, loss: 0.08414
Validation Loss:  0.1923576295375824
Epoch: 4, loss: 0.08338
Validation Loss:  0.19332800805568695
Epoch: 5, loss: 0.08264
Validation Loss:  0.19425222277641296
Epoch: 6, loss: 0.08190
Validation Loss:  0.19518983364105225
Epoch: 7, loss: 0.08116
Validation Loss:  0.19610872864723206
Epoch: 8, loss: 0.08043
Validation Loss:  0.19695301353931427
Epoch: 9, loss: 0.07970
Validation Loss:  0.197836771607399
Epoch: 10, loss: 0.07899
Validation Loss:  0.19868560135364532
Epoch: 11, loss: 0.07827
Validation Loss:  0.19948600232601166
Epoch: 12, loss: 0.07756
Validation Loss:  0.2002599984407425
Epoch: 13, loss: 0.07685
Validation Loss:  0.2009965479373932
Epoch: 14, loss: 0.07615
Validation Loss:  0.2016681581735611
Epoch: 15, loss: 0.07544
Validation Loss:  0.20225955545902252
Epoch: 1

In [ ]:
plt.plot(train_loss, label='train_loss')
plt.plot(val_loss, label='val_loss')
# plt.yticks(np.arange(0, 0.001, 1e-6))
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
if(save == True):
  plt.savefig(results_path + 'loss_curve.jpg', bbox_inches='tight', facecolor='w')
plt.show()


In [ ]:
X_test1 = Variable(torch.Tensor(X_test)) #converting to Tensors
y_test1 = Variable(torch.Tensor(y_test))
#reshaping the dataset
X_test1 = torch.reshape(X_test1, (X_test1.shape[0], 1, X_test1.shape[1]))

In [ ]:
test_predict = lstm1(X_test1)#forward pass
data_predict = test_predict.data.numpy()#numpy conversion
dataY_plot = y_test1.data.numpy()

data_predict = mm.inverse_transform(data_predict) #reverse transformation
dataY_plot = mm.inverse_transform(dataY_plot)

# data_predict = [x - (0.02e-5) for x in data_predict]
plt.figure(figsize=(20,6)) #plotting
plt.axvline(x=200, c='r', linestyle='--') #size of the training set
# ax.set_ylim([ymin, ymax])

# plt.xlim([1, ])
# plt.ylim([1.2, 1.8])
# plt.yticks(np.arange(1.2, 1.8, 0.2))
# plt.xticks(np.arange(-3, 3, 0.25))
plt.plot(data_predict, label='Predicted Data [Test]') #predicted plot
plt.plot(dataY_plot, label='Actual Data [Test]') #actual plot
plt.title('Time-Series Prediction')
# plt.ylim((2.2e-5, 2.8e-5))
plt.legend()
plt.xlabel("Series")
plt.ylabel("Price")
if(save == True):
  plt.savefig(results_path + 'price_preds_test.jpg', bbox_inches='tight', facecolor='w')
plt.show()

# print(len(d))

In [ ]:
#Full plot
data_train = mm.inverse_transform(y_train)
x = [i for i in range(len(data_train))]
last_x = x[len(x) - 1]
x2 = [i for i in range(last_x + 1, last_x + 1 + len(data_predict))]

# x.extend(x2)
plt.figure(figsize=(20,6))
plt.plot(x, data_train, color='blue', label='Actual Price [Train]')
plt.plot(x2, data_predict, color='red', label='Predicted Price')
plt.plot(x2, dataY_plot, color='yellow', label='Actual Price [Test]')
plt.legend()
plt.ylabel('Price')
plt.xlabel('Datapoint index')
if(save == True):
  plt.savefig(results_path + 'price_curve_whole.jpg', bbox_inches='tight', facecolor='w')
plt.show()



In [ ]:
mses = []
maes = []
# rmses = 
for i in range(len(data_predict)):
  mse = (data_predict[i] - dataY_plot[i])**2
  mae = abs(data_predict[i] - dataY_plot[i])
  # print(f"pred = {data_predict[i]} | true = {dataY_plot[i]} | MSE = {mse}")
  mses.append(mse)
  maes.append(mae)
plt.plot([x for x in range(len(mses))], mses)
plt.show()

In [ ]:
test_rmse = math.sqrt(sum(mses) / len(mses))
test_mae = (sum(maes) / len(maes))[0]
print(f"Test RMSE = {test_rmse}")
print(f"Test MAE = {test_mae}")

In [ ]:
perc_errors = []
mean = 0
for i in range(len(data_predict)):
  mae_perc = ((abs(data_predict[i] - dataY_plot[i])) / dataY_plot[i] ) * 100
  perc_errors.append(mae_perc)
  mean += mae_perc

In [ ]:
perc_errors = np.array(perc_errors)
mean_perc_error = np.mean(perc_errors)
max_perc_error = np.amax(perc_errors)
print(f"Mean error = {mean_perc_error}")
print(f"max : {max_perc_error}")
# print(perc_errors.shape)

In [ ]:
print('R2 Score : ', metrics.r2_score(data_predict, dataY_plot))

In [ ]:
if(save == True):
  stats_f = open(results_path + 'stats.txt', 'w')
  lines = [f'Stats for Crypto {crypto}\n',
          f'Model params : \n Hidden Layer size = {hidden_size}\n Learning_Rate = {learning_rate}\n Num epochs = {num_epochs}\n',
          f'Results : \n Test MAE = {test_mae}\n Test RMSE = {test_rmse} \n Mean percentage error = {mean_perc_error} \n Max perc error = {max_perc_error}'
          ]
  stats_f.writelines(lines)
  stats_f.close()